In [1]:
# Optymalizację możemy zacząć już na etapie procesu wczytywania danych do pandas DataFrame.
import pandas as pd
import numpy as np
from datetime import datetime

# wczytywanie danych z pliku "na raz" - przy małych plikach optymalne rozwiązanie
df = pd.read_csv('zamowienia.csv', header=0, sep=';')
display(df.head())
# poniższa funkcja zwróci nam między innymi typy danych dla każdej kolumny i SZACUNKOWĄ wielkość pamięci, którą zajmuje
# nie jest to jednak wielkość dokładna
df.info()

new_df = pd.concat([df.sample(frac=1) for n in range(50_000)])
new_df.to_csv('zamowienia_expanded.csv', header=True, index=False)



,Kraj,Sprzedawca,Data zamowienia,idZamowienia,Utarg
0,Polska,Kowalski,2003-07-16,10248,440.00
1,Polska,Sowiński,2003-07-10,10249,1863.40
2,Niemcy,Peacock,2003-07-12,10250,1552.60
3,Niemcy,Leverling,2003-07-15,10251,654.06
4,Niemcy,Peacock,2003-07-11,10252,3597.90


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 799 entries, 0 to 798
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Kraj             799 non-null    object 
 1   Sprzedawca       799 non-null    object 
 2   Data zamowienia  799 non-null    object 
 3   idZamowienia     799 non-null    int64  
 4   Utarg            799 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 31.3+ KB
